In [2]:
# データ加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
sns.set()

# 機械学習ライブラリ
import sklearn

# 小数第3位まで表示
%precision 3

ModuleNotFoundError: No module named 'pandas'

In [20]:
df = pd.read_csv('../Data/train.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['Month'] = df['datetime'].dt.month
df['Day'] = df['datetime'].dt.day
df = df.drop('datetime', axis=1)
df.replace(-1, pd.np.nan, inplace=True)
df = df.interpolate(axis=1)
df.head(40)
y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values

In [21]:
from sklearn import preprocessing

sscaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)
sscaler.fit(X)
X = sscaler.transform(X)
mmscaler = preprocessing.MinMaxScaler() # インスタンスの作成
mmscaler.fit(X)           # xの最大・最小を計算
X = mmscaler.transform(X) # xを変換

In [22]:
df_ = pd.read_csv('../Data/test.csv')
df_.head(10)
df_['datetime'] = pd.to_datetime(df_['datetime'])
df_['Month'] = df_['datetime'].dt.month
df_['Day'] = df_['datetime'].dt.day
df_date = df_['datetime']
df_ = df_.drop('datetime', axis=1)
df_.replace(-1, pd.np.nan, inplace=True)
df_ = df_.interpolate(axis=1)
X_ = df_.iloc[:, :].values
sscaler.fit(X_)
X_ = sscaler.transform(X_)
mmscaler.fit(X_)           # xの最大・最小を計算
X_ = mmscaler.transform(X_) # xを変換

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
# X_train = X
# y_train = y

In [24]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [25]:
model = GradientBoostingRegressor(random_state=0)
model_name = 'GradientBoosting'
model.fit(X_train, y_train)
score = {}
score[(model_name, 'train_score')] = model.score(X_train, y_train)
# score[(model_name, 'test_score')] = model.score(X_test, y_test)
pd.Series(score).unstack()
out = model.predict(X_)
out = pd.DataFrame({'datetime': df_date,'y':out})
out.to_csv('../Data/answer.csv', header=False, index=False)

In [26]:
models = {
   'tree': DecisionTreeRegressor(random_state=0),
    'AdaBoost': AdaBoostRegressor(DecisionTreeRegressor(random_state=0), random_state=0),
    'Bagging': BaggingRegressor(DecisionTreeRegressor(random_state=0), n_estimators=100, random_state=0),
    'Bagging & AdaBoost': AdaBoostRegressor(BaggingRegressor(DecisionTreeRegressor(random_state=0), n_estimators=100,random_state=0),random_state=0),
    'RandomForest': RandomForestRegressor(random_state=0),
    'GradientBoost': GradientBoostingRegressor(random_state=0)
}
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    # scores[(model_name, 'test_score')] = model.score(X_test, y_test)

pd.Series(scores).unstack()

,train_score
AdaBoost,0.820809
Bagging,0.841257
Bagging & AdaBoost,0.812264
GradientBoost,0.618738
RandomForest,0.841322
tree,0.878287


In [27]:
import os
import random

def reset_seed(seed=0):
    
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [28]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.4403192598760036
0.4498477497298524


In [29]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=0)
xgb_model.fit(X_train, y_train)

out = xgb_model.predict(X_)
# mse=mean_squared_error(y_test, out)
print(np.sqrt(mse))


# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)
# xgb_params = {
#         # 回帰問題
#         'objective': 'reg:linear',
#         # 学習用の指標 (RMSE)
#         'eval_metric': 'rmse',
#     }
# evals = [(dtrain, 'train'), (dtest, 'eval')]
# eval_result = {}
# bst = xgb.train(xgb_params,
#                     dtrain,
#                     num_boost_round=1000,
#                     early_stopping_rounds=100,
#                     evals=evals,
#                     evals_result=eval_result,)
# out = bst.predict(dtest)
# mse = mean_squared_error(y_test, y_pred)
# print('RMSE:', math.sqrt(mse))


[09:53:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[09:53:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


NameError: name 'mse' is not defined

In [32]:
out = pd.DataFrame({'datetime': df_date,'y':out})
out.to_csv('../Data/xgb.csv', header=False, index=False)

ValueError: Shape of passed values is (2, 2), indices imply (365, 2)

In [33]:
import tensorflow as tf 
import numpy as np 
from tensorflow.keras import Sequential 
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [37]:
model = Sequential()

model.add(layers.LSTM(50, input_shape=(10, 6), activation='relu'))
model.add(layers.Dense(1, activation='relu'))

optimizer = optimizers.Adam(lr=0.01)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

model.fit(
    X_train,
    y_train,
    batch_size=30,
    epochs=10,
    validation_split=0.1
)

Epoch 1/10


ValueError: in user code:

    C:\Users\SaToShi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\SaToShi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\SaToShi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\SaToShi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\SaToShi\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(